[View in Colaboratory](https://colab.research.google.com/github/joheras/CLoDSA/blob/master/notebooks/CLODSA_Stack_Segmentation.ipynb)

# Augmenting a dataset for segmentation in stacks of images

In this notebook, we illustrate how CLODSA can be employed to augment a dataset of stacks of images for segmentation. In particular, we use the [HL60 Cell Line dataset](https://cbia.fi.muni.cz/datasets/). A subset of that dataset can be downloaded using the following command. 

In [16]:
!wget https://www.dropbox.com/s/1wi6sr1noexmrjb/HL60.zip?dl=0 -O HL60.zip
!unzip HL60.zip

--2019-01-22 13:40:52--  https://www.dropbox.com/s/1wi6sr1noexmrjb/HL60.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.1, 2620:100:6024:1::a27d:4401
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.1]:443... conectado.
Petición HTTP enviada, esperando respuesta... 301 Moved Permanently
Ubicación: /s/raw/1wi6sr1noexmrjb/HL60.zip [siguiente]
--2019-01-22 13:40:52--  https://www.dropbox.com/s/raw/1wi6sr1noexmrjb/HL60.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://ucdb5e68dfc91b9d740d4a6de409.dl.dropboxusercontent.com/cd/0/inline/AZ6_Jq6GgTLrPgRlSFVhMUOTlAW0-x209uFlA8b7lkujC4bZRmOpyt1XYOjgVPH0xukFyBBOyM1C1KB0-9aNH4GOCBE8zd7mRN9qqo6YEorLbedqaDSwxNCM_MVCEhA7JNxCD8imeC82k9F6YpnsCsv5NfoQWYBj2OFNR2EVBgC62oH-RPU8thHoFzTMm6xSiTI/file# [siguiente]
--2019-01-22 13:40:52--  https://ucdb5e68dfc91b9d740d4a6de409.dl.dropboxusercontent.com/cd/0/inline/AZ6_Jq6GgTLrPgRlSFVhMUOTlAW0-x209uFlA8b

We can check the amount of stack of images in the folder.

In [17]:
print("HL60 images")
!ls HL60/images/
print("HL60 labels")
!ls HL60/labels/

HL60 images
0000.tif  0001.tif  0002.tif
HL60 labels
0000.tif  0001.tif  0002.tif


## Augmentation techniques

In this notebook, we illustrate how the following augmentation techniques can be applied to the downloaded dataset:
- Horizontal flip.
- Average blurring.

## Installing the necessary libraries

In case that CLODSA is not installed in your system, the first task consists in installing it using ``pip``.

In [ ]:
!pip install clodsa

## Loading the necessary libraries

The first step in the pipeline consists in loading the necessary libraries to apply the data augmentation techniques in CLODSA.

In [1]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import cv2

Using TensorFlow backend.


## Showing the stacks

We include the functionality to show a stack of images stored in a tif file as an annimation. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML
import imageio

def animateTiff(tifPath):
    plt.ioff()
    fig = plt.figure()
    ims = imageio.mimread(tifPath)
    an_ims = []
    for i in range(len(ims)):
        im = plt.imshow(ims[i], animated=True)
        an_ims.append([im])
    anim = animation.ArtistAnimation(fig, an_ims, interval=20, blit=True,
                                    repeat_delay=1000)
    return(anim.to_html5_video())

An example.

In [3]:
HTML(animateTiff("HL60/images/0000.tif"))

/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functionality might be degraded or be slow.

  wa

In [4]:
anim2=animateTiff("HL60/labels/0000.tif")
HTML(anim2)

/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7472: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)


## Creating the augmentor object

As explained in the documentation of CLODSA, we need to specify some parameters for the augmentation process, and use them to create an augmentor object.  

_The kind of problem_. In this case, we are working in a stacksegmentation problem

In [5]:
PROBLEM = "stacksegmentation"

_The annotation mode_. The annotation is youtubevideo. 

In [6]:
ANNOTATION_MODE = "tiff"

_The input path_. The input path containing the videos. 

In [7]:
INPUT_PATH = "HL60/"

_The generation mode_. In this case, linear, that is, all the augmentation techniques are applied to all the stacks of the original dataset. 

In [9]:
GENERATION_MODE = "linear"

_The output mode_. The generated videos will be stored in a new folder called augmented_videos_youtube. We must also provide the name of the csv with the annotations. That file must be in the same folder as the videos. 

In [10]:
OUTPUT_MODE = "tiff"
OUTPUT_PATH= "augmented_HL60/"

Using the above information, we can create our augmentor object. 

In [11]:
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH})

## Adding the augmentation techniques

Now, we define the techniques that will be applied in our augmentation process and add them to our augmentor object. 



First of all, we must define a transformer generator.

In [12]:
transformer = transformerGenerator(PROBLEM)

#### Horizontal flip

In [13]:
hFlip = createTechnique("flip",{"flip":1})
augmentor.addTransformer(transformer(hFlip))

**Average blurring**

In [14]:
avgBlur = createTechnique("average_blurring", {"kernel" : 5})
augmentor.addTransformer(transformer(avgBlur))

#### None
(to keep also the original stack)

In [16]:
none = createTechnique("none",{})
augmentor.addTransformer(transformer(none))

## Applying the augmentation process

Finally, we apply the augmentation process (this might take some time depending on the number of videos of the original dataset and the number of transformations that will be applied). 

In [17]:
augmentor.applyAugmentation()

/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7472: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7472: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7472: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops c

Finally, we can check the amount of videos in the output folder.

In [18]:
print("Number of stacks in folder")
!ls augmented_HL60/images | wc -l

Number of stacks in folder
12


We can compare the original stack with one of the generated (for instance, one where we have applied the flip operation).

Original

In [19]:
HTML(animateTiff("HL60/images/0000.tif"))

Flip version.

In [20]:
HTML(animateTiff("augmented_HL60/images/2_0_0000.tif"))

Original annotation.

In [22]:
HTML(animateTiff("HL60/labels/0000.tif"))

/home/jonathan/.virtualenvs/cv/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7472: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)


Generated annotation. 

In [23]:
HTML(animateTiff("augmented_HL60/labels/2_0_0000.tif"))

If you are executing this notebook in Colaboratory, you need to download the generated files. To that aim, you can create a zip folder and download it using the following commands. 

In [ ]:
!zip -r augmented_HL60.zip augmented_HL60
from google.colab import files
files.download('augmented_HL60.zip')